In [1]:
!pip install pyspark==3.5.3

In [39]:
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_1.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_2.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_3.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_4.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_5.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_6.log.gz
!wget -nc https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_7.log.gz

--2024-11-05 18:55:01--  https://raw.githubusercontent.com/keeyong/sjsu-data226/refs/heads/main/week12/data/sample_web_log_1.log.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10277393 (9.8M) [application/octet-stream]
Saving to: ‘sample_web_log_1.log.gz’

sample_web_log_1.lo 100%[===================>]   9.80M  32.2MB/s    in 0.3s    

2024-11-05 18:55:02 (32.2 MB/s) - ‘sample_web_log_1.log.gz’ saved [10277393/10277393]

File ‘sample_web_log_2.log.gz’ already there; not retrieving.

File ‘sample_web_log_3.log.gz’ already there; not retrieving.

File ‘sample_web_log_4.log.gz’ already there; not retrieving.

File ‘sample_web_log_5.log.gz’ already there; not retrieving.

File ‘sample_web_log_6.log.gz’ already there; not retrieving.

File ‘sample_web_log_7.l

## Let's set up Spark Session and create an input dataframe

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("HandleLogFiles").getOrCreate()

In [ ]:
# Load all .gz files in the directory into a DataFrame
df = spark.read.text("*.gz")

In [40]:
# Check the number of partitions
print(df.rdd.getNumPartitions())

df.show(truncate=False)

2
+-----------------------------------------------------------------------------------+
|value                                                                              |
+-----------------------------------------------------------------------------------+
|123.45.67.89 - - [05/Nov/2024:02:08:16 +0000] "DELETE /cart HTTP/1.1" 500 242      |
|192.168.1.1 - - [04/Nov/2024:21:23:39 +0000] "POST /checkout HTTP/1.1" 404 2781    |
|234.56.78.90 - - [05/Nov/2024:07:06:19 +0000] "GET /api/data HTTP/1.1" 301 3758    |
|192.168.1.1 - - [04/Nov/2024:20:03:56 +0000] "POST /home HTTP/1.1" 200 1837        |
|192.168.1.1 - - [04/Nov/2024:21:25:05 +0000] "GET /products/123 HTTP/1.1" 200 3430 |
|234.56.78.90 - - [04/Nov/2024:07:38:10 +0000] "GET /api/data HTTP/1.1" 404 3729    |
|123.45.67.89 - - [04/Nov/2024:12:33:22 +0000] "PUT /api/data HTTP/1.1" 404 799     |
|192.168.1.1 - - [04/Nov/2024:07:37:46 +0000] "GET /api/data HTTP/1.1" 500 309      |
|123.45.67.89 - - [04/Nov/2024:21:52:36 +0000] "POST

In [ ]:
# Extract the necessary information from log data using regular expressions
pattern = r'(\d+\.\d+\.\d+\.\d+) - - \[(.*?)\] "(.*?) (.*?) HTTP.*" (\d+) (\d+)'

log_df = df.select(
    F.regexp_extract("value", pattern, 1).alias("ip"),
    F.regexp_extract("value", pattern, 2).alias("timestamp"),
    F.regexp_extract("value", pattern, 3).alias("method"),
    F.regexp_extract("value", pattern, 4).alias("url"),
    F.regexp_extract("value", pattern, 5).alias("status").cast("integer"),
    F.regexp_extract("value", pattern, 6).alias("size").cast("integer")
)

In [29]:
log_df.show()

+------------+--------------------+------+-------------+------+----+
|          ip|           timestamp|method|          url|status|size|
+------------+--------------------+------+-------------+------+----+
|123.45.67.89|05/Nov/2024:02:08...|DELETE|        /cart|   500| 242|
| 192.168.1.1|04/Nov/2024:21:23...|  POST|    /checkout|   404|2781|
|234.56.78.90|05/Nov/2024:07:06...|   GET|    /api/data|   301|3758|
| 192.168.1.1|04/Nov/2024:20:03...|  POST|        /home|   200|1837|
| 192.168.1.1|04/Nov/2024:21:25...|   GET|/products/123|   200|3430|
|234.56.78.90|04/Nov/2024:07:38...|   GET|    /api/data|   404|3729|
|123.45.67.89|04/Nov/2024:12:33...|   PUT|    /api/data|   404| 799|
| 192.168.1.1|04/Nov/2024:07:37...|   GET|    /api/data|   500| 309|
|123.45.67.89|04/Nov/2024:21:52...|  POST|    /checkout|   301|2375|
|123.45.67.89|04/Nov/2024:08:36...|DELETE|    /api/data|   404|3449|
| 192.168.1.1|05/Nov/2024:03:15...|   GET|    /api/data|   200|2319|
|234.56.78.90|05/Nov/2024:01:26...

## Let's compute top 404 urls

In [24]:
# Keep only 404 error logs
error_404_logs = log_df.filter(log_df.status == 404)

In [25]:
# Group by URL and then count, and sort by count in descending order
url_404_count = error_404_logs.groupBy("url").count().orderBy(F.desc("count"))

In [26]:
# print the outcome
url_404_count.show()

+-------------+------+
|          url| count|
+-------------+------+
|/products/123|300653|
|        /home|300048|
|    /api/data|299683|
|        /cart|299543|
|    /checkout|299506|
+-------------+------+



## Now Let's do this in SparkSQL

In [30]:
# Register the DataFrame as a temporary SQL table
log_df.createOrReplaceTempView("logs")

In [31]:
# Use SparkSQL to count URLs with 404 status
url_404_count = spark.sql("""
    SELECT url, COUNT(*) as count
    FROM logs
    WHERE status = 404
    GROUP BY url
    ORDER BY count DESC
""")

In [32]:
url_404_count.show()

+-------------+------+
|          url| count|
+-------------+------+
|/products/123|300653|
|        /home|300048|
|    /api/data|299683|
|        /cart|299543|
|    /checkout|299506|
+-------------+------+



## Let's save this DF (url_404_count) as a table in Snowflake

In [2]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar

--2024-11-05 18:20:38--  https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.19.0/snowflake-jdbc-3.19.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70986770 (68M) [application/java-archive]
Saving to: ‘snowflake-jdbc-3.19.0.jar’

snowflake-jdbc-3.19 100%[===================>]  67.70M  22.9MB/s    in 3.0s    

2024-11-05 18:20:41 (22.9 MB/s) - ‘snowflake-jdbc-3.19.0.jar’ saved [70986770/70986770]



In [34]:
from google.colab import userdata

account = userdata.get('snowflake_account')
user = userdata.get('snowflake_userid')
password = userdata.get('snowflake_password')
database = "dev"
schema = "analytics"

url = f"jdbc:snowflake://{account}.snowflakecomputing.com/?db={database}&schema={schema}&user={user}&password={password}"

In [37]:
url_404_count.write \
    .format("jdbc") \
    .option("driver", "net.snowflake.client.jdbc.SnowflakeDriver") \
    .option("url", url) \
    .option("dbtable", "url_404_count") \
    .mode("overwrite") \
    .save()

In [ ]:
# Now go back to Snowflake and check the table (analytics.url_404_count)